# Daily Seed Tweet Processing
Iterate all unprocessed tweets from seed users (processed tweets are marked with `processed_seed=True`) and upsert them with `depth=1`, namely:
* all **mentioned** users
* all **authors** of retweeted tweets
* all **authors** of quoted tweets (retweet + comment)
* all **authors** of replied tweets

In [ ]:
# required imports to access api_db, misc, misc.CONFIG, ...
import sys
sys.path = ['.', '..', '../..'] + sys.path
from collection import *

### Conditional Execution
Each file needs to verify if it should be executed or not based on the configurations (for some files this is not optional but all should have this section, even if it is tautological). Example:
```python
if not misc.CONFIG["collection"]["execute_this_script"]: exit()
```

In [ ]:
# Conditional execution
pass

<hr>
<h1 align="center">driver code</h1>

In [ ]:
from pytictoc import TicToc

In [ ]:
processed_key = "processed_seed"

In [ ]:
with TicToc():
    print("Loading seed_ids from database...", end="", flush=True)
    seed_ids = [s["_id"] for s in api_db.col_users.find({"depth": 0}, {}).limit(len(misc.CONFIG["seed"]["usernames"]))]
    print("got %d seed users, done." % len(seed_ids))

In [ ]:
find_params = {processed_key: {"$exists": False}, "user": {"$in": seed_ids}}
retrieve_params = {"user_mentions": True, "user": True, "retweeted_status.user" :True, "quoted_status.user": True, "in_reply_to_user_id": True}
tweets = api_db.col_tweets.find(find_params, retrieve_params, no_cursor_timeout=True)

print("Processing seed tweets (silent db-only)...", end="", flush=True)
count = 0
with TicToc():
    for t in tweets:
        # fill user_ids list from differnt tweet information
        user_ids = [t["user"]]
        if "user_mentions" in t: user_ids.extend(t["user_mentions"])
        if "retweeted_status" in t: user_ids.append(t["retweeted_status"]["user"])
        if "quoted_status" in t: user_ids.append(t["quoted_status"]["user"])
        if "in_reply_to_user_id" in t: user_ids.append(t["in_reply_to_user_id"])
        # remove duplicates, update user counts, mark tweet as procesed
        user_ids = list(set(user_ids)) # they should not count twice
        insert_user_ids(user_ids, depth=1)
        upsert_tweet_info({"_id": t["_id"], processed_key: True})
        count+=1
    print("processed %d new tweets, done." % (count))

In [ ]:
print("DONE")